<a href="https://colab.research.google.com/github/dacardonave/Modelo_Desercion_Clientes_De_Una_Empresa/blob/main/Modelo_Desercion_Clientes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prueba tecnica Ingeniero de Datos – Comercial Nutresa 2021

### Daniel Cardona Velasquez

## Planteamiento del reto

###### La compañía para la cual trabaja desea contar con una herramienta que le permita identificar anticipadamente los clientes que se van a retirar en el siguiente periodo, con el objetivo de implementar planes de acción que logren reducir este comportamiento tanto como sea posible. Por este motivo, se le ha encargado construir un modelo que a partir de la base de datos suministrada permita predecir, con la mayor precisión posible, la variable “Retiro”.

###### Para cumplir con este reto debe dar respuesta a las preguntas y lineamientos que se indican a continuación:

* Realice un análisis descriptivo de los datos, depure y seleccione las variables que considere más importantes.
*	Defina que indicador debe usar para medir la precisión del modelo y la metodología de validación a seguir.
*	Entrene y valide los modelos que considere necesarios y seleccione el mejor con base en sus criterios.
*	Plantee un plan de trabajo de puesta en marcha y administración del modelo.
*	Envíe un archivo Markdown o Jupyter Notebook que responda a cada uno de los puntos anteriores, debidamente argumentado y en el cual se explique el paso a paso realizado.


### Librerias

In [2]:
 #Para admitir python 2 y python 3
from __future__ import division, print_function, unicode_literals

# Importo las librerias a usar
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pickle as pk
import numpy as np
from sklearn.metrics import confusion_matrix
import joblib
import os
from joblib import dump
from sklearn.naive_bayes import GaussianNB

#Ignora las advertencias inútiles
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# plot
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colors

### Importacion de los datos y lectura de los datos


In [17]:
BD = pd.read_csv('Data.csv', sep=";")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,16,20,21,24,25,28,29,36,42,45,47,55,56,57,62) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Datos 

Se observan 63 variables y la etiqueta de salida (Retiro) a predecir

In [23]:
BD.head(10)

,Id,Lifetime,user_intake,user_no_outgoing_activity_in_days,user_account_balance_last,user_spendings,user_has_outgoing_calls,user_has_outgoing_sms,user_use_gprs,user_does_reload,reloads_inactive_days,reloads_count,reloads_sum,calls_outgoing_count,calls_outgoing_spendings,calls_outgoing_duration,calls_outgoing_spendings_max,calls_outgoing_duration_max,calls_outgoing_inactive_days,calls_outgoing_to_onnet_count,calls_outgoing_to_onnet_spendings,calls_outgoing_to_onnet_duration,calls_outgoing_to_onnet_inactive_days,calls_outgoing_to_offnet_count,calls_outgoing_to_offnet_spendings,calls_outgoing_to_offnet_duration,calls_outgoing_to_offnet_inactive_days,calls_outgoing_to_abroad_count,calls_outgoing_to_abroad_spendings,calls_outgoing_to_abroad_duration,calls_outgoing_to_abroad_inactive_days,sms_outgoing_count,sms_outgoing_spendings,sms_outgoing_spendings_max,sms_outgoing_inactive_days,sms_outgoing_to_onnet_count,sms_outgoing_to_onnet_spendings,sms_outgoing_to_onnet_inactive_days,sms_outgoing_to_offnet_count,sms_outgoing_to_offnet_spendings,sms_outgoing_to_offnet_inactive_days,sms_outgoing_to_abroad_count,sms_outgoing_to_abroad_spendings,sms_outgoing_to_abroad_inactive_days,sms_incoming_count,sms_incoming_spendings,sms_incoming_from_abroad_count,sms_incoming_from_abroad_spendings,gprs_session_count,gprs_usage,gprs_spendings,gprs_inactive_days,last_100_reloads_count,last_100_reloads_sum,last_100_calls_outgoing_duration,last_100_calls_outgoing_to_onnet_duration,last_100_calls_outgoing_to_offnet_duration,last_100_calls_outgoing_to_abroad_duration,last_100_sms_outgoing_count,last_100_sms_outgoing_to_onnet_count,last_100_sms_outgoing_to_offnet_count,last_100_sms_outgoing_to_abroad_count,last_100_gprs_usage,Retiro
0,13,1000,0,1,"0,05",0,1,1,0,0,66,0,0,27,0,"1,02",0,"0,58",1,2,0,0,1,6,0,0,1,0,0,0,1,6,0,0,5,0,0,5,5,0,5,0,0,5,18,0,0,0,0,0,0,1276,4,"12,01","75,27",0,"63,43",0,210,1,84,0,0,0
1,14,1000,0,25,"28,31","3,45",1,0,0,0,1276,0,0,5,"3,45","9,9","1,51","4,33",25,5,"3,45","9,9",25,0,0,0,25,5,"3,45","9,9",25,0,0,0,1276,0,0,1276,0,0,1276,0,0,1276,0,0,0,0,0,0,0,1276,0,0,"13,38","11,18",2,"11,18",0,0,0,0,0,0
2,18,1005,0,8,"15,62","1,97",1,0,0,0,1276,0,0,18,"1,97","14,92","0,89","9,23",8,0,0,0,8,0,0,0,8,6,"1,31","10,45",8,0,0,0,1276,0,0,1276,0,0,1276,0,0,1276,7,0,4,0,0,0,0,1276,0,0,30,0,0,"10,45",0,0,0,0,0,0
3,27,1013,0,11,"5,62",0,1,0,0,0,1276,0,0,1,0,0,0,0,11,0,0,0,11,0,0,0,11,0,0,0,11,0,0,0,1276,0,0,1276,0,0,1276,0,0,1276,0,0,0,0,0,0,0,1276,0,0,0,0,0,0,0,0,0,0,0,1
4,32,1032,0,2,"5,86","0,15",1,0,0,1,17,1,6,22,"0,15","2,58","0,15",1,2,0,0,0,2,4,"0,15",1,2,0,0,0,2,0,0,0,1276,0,0,1276,0,0,1276,0,0,1276,5,0,0,0,0,0,0,1276,1,6,"2,58",0,1,0,0,0,0,0,0,0
5,35,1039,0,1,"12,51","12,96",1,1,0,1,6,1,17,56,"12,96","90,73","2,39","15,95",1,0,0,0,1,15,"8,44","56,28",1,0,0,0,1,2,0,0,6,0,0,6,0,0,6,0,0,6,11,0,0,0,0,0,0,1276,3,29,"356,65",0,"215,83",0,4,1,1,0,0,0
6,36,1040,0,1,"16,73","2,34",1,1,0,1,4,1,12,8,0,0,0,0,5,0,0,0,5,0,0,0,5,0,0,0,5,41,"2,34","0,06",1,0,0,1,39,"2,34",1,0,0,1,4,0,0,0,0,0,0,1276,1,12,0,0,0,0,112,0,109,0,0,0
7,47,521,0,1,"0,66","45,89",1,1,0,1,2,11,48,395,"32,93","253,33","2,3","15,35",1,0,0,0,1,129,"25,89","172,6",1,0,0,0,1,246,"12,96","0,06",1,0,0,1,215,"12,72",1,0,0,1,107,0,0,0,0,0,0,1276,35,158,"735,87",0,"473,5",0,1123,0,1006,0,0,0
8,52,824,0,2,"4,49","1,37",1,1,0,1,27,1,"12,02",20,"1,37","10,77","0,4","2,68",4,0,0,0,4,14,"1,22","8,1",4,0,0,0,4,36,0,0,2,0,0,2,0,0,2,0,0,2,10,6,0,0,0,0,0,57,6,"30,03","56,22",0,"48,02",0,122,1,1,0,"2,77",0
9,59,867,0,2,"90,93","17,99",1,1,0,1,12,1,"72,06",88,"17,99","121,53","4,09","27,27",2,0,0,0,2,14,"17,99","119,95",2,0,0,0,2,16,0,0,4,0,0,4,16,0,4,0,0,4,3,0,0,0,0,0,0,1276,7,"108,06","587,85",0,"585,55",0,19,0,19,0,0,0


## Análisis descriptivo y exploratorio


In [35]:
# Tamaño del dataset
print(BD.shape)

(66469, 64)


In [28]:
# Informacion sobre tipo y cantidad de los datos
BD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66469 entries, 0 to 66468
Data columns (total 64 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Id                                          66469 non-null  int64 
 1   Lifetime                                    66469 non-null  int64 
 2   user_intake                                 66469 non-null  int64 
 3   user_no_outgoing_activity_in_days           66469 non-null  int64 
 4   user_account_balance_last                   66469 non-null  object
 5   user_spendings                              66469 non-null  object
 6   user_has_outgoing_calls                     66469 non-null  int64 
 7   user_has_outgoing_sms                       66469 non-null  int64 
 8   user_use_gprs                               66469 non-null  int64 
 9   user_does_reload                            66469 non-null  int64 
 10  reloads_inactive_days 

In [33]:
# Estadisticas basica de las vairbales
BD.describe()

,Id,Lifetime,user_intake,user_no_outgoing_activity_in_days,user_has_outgoing_calls,user_has_outgoing_sms,user_use_gprs,user_does_reload,reloads_inactive_days,reloads_count,calls_outgoing_count,calls_outgoing_inactive_days,calls_outgoing_to_onnet_count,calls_outgoing_to_onnet_inactive_days,calls_outgoing_to_offnet_count,calls_outgoing_to_offnet_inactive_days,calls_outgoing_to_abroad_count,calls_outgoing_to_abroad_inactive_days,sms_outgoing_count,sms_outgoing_inactive_days,sms_outgoing_to_onnet_count,sms_outgoing_to_onnet_inactive_days,sms_outgoing_to_offnet_count,sms_outgoing_to_offnet_inactive_days,sms_outgoing_to_abroad_count,sms_outgoing_to_abroad_inactive_days,sms_incoming_count,sms_incoming_from_abroad_count,gprs_session_count,gprs_inactive_days,last_100_reloads_count,last_100_sms_outgoing_count,last_100_sms_outgoing_to_onnet_count,last_100_sms_outgoing_to_offnet_count,last_100_sms_outgoing_to_abroad_count,Retiro
count,6.646900e+04,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000,66469.000000
mean,5.948572e+05,4006.037867,0.040365,63.975718,0.804450,0.603229,0.045074,0.560246,361.313515,0.934917,35.334562,197.809686,0.317170,197.809686,16.312913,197.809686,1.477937,197.809686,34.366622,411.987408,2.500038,411.987408,21.491583,411.987408,1.016775,411.987408,8.954385,0.797560,6.329492,1187.918097,3.347500,101.418105,7.318630,62.272684,3.062616,0.209225
std,4.292423e+05,6058.350935,0.196815,264.006882,0.396626,0.489232,0.207467,0.496361,556.759885,1.344173,63.591271,461.054350,2.404746,461.054350,33.851877,461.054350,12.625820,461.054350,91.695821,595.994223,13.071417,595.994223,66.754785,595.994223,7.331246,595.994223,14.220569,3.127624,94.147863,334.434914,6.014605,263.280424,35.573779,185.571343,25.071290,0.406759
min,1.300000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.501070e+05,457.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,8.000000,0.000000,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,2.000000,0.000000,2.000000,0.000000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1276.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.135120e+05,1216.000000,0.000000,2.000000,1.000000,1.000000,0.000000,1.000000,25.000000,1.000000,15.000000,3.000000,0.000000,3.000000,4.000000,3.000000,0.000000,3.000000,3.000000,10.000000,0.000000,10.000000,1.000000,10.000000,0.000000,10.000000,6.000000,0.000000,0.000000,1276.000000,1.000000,14.000000,0.000000,5.000000,0.000000,0.000000
75%,1.041120e+06,1985.000000,0.000000,8.000000,1.000000,1.000000,0.000000,1.000000,1276.000000,1.000000,43.000000,15.000000,0.000000,15.000000,18.000000,15.000000,0.000000,15.000000,30.000000,1276.000000,0.000000,1276.000000,15.000000,1276.000000,0.000000,1276.000000,12.000000,0.000000,0.000000,1276.000000,4.000000,96.000000,1.000000,51.000000,0.000000,0.000000
max,1.196944e+06,15947.000000,1.000000,1307.000000,1.000000,1.000000,1.000000,1.000000,1338.000000,28.000000,2495.000000,1338.000000,191.000000,1338.000000,1319.000000,1338.000000,2326.000000,1338.000000,3391.000000,1338.000000,922.000000,1338.000000,2940.000000,1338.000000,479.000000,1338.000000,716.000000,306.000000,9024.000000,1338.000000,236.000000,18637.000000,2829.000000,13860.000000,4692.000000,1.000000


In [38]:
#conteo de los valores unicos de la etiqueta a predecir 
BD["Retiro"].value_counts() 

0    52562
1    13907
Name: Retiro, dtype: int64

# Matriz de Correlacion

La correlacion permite hacer inferencias a partir de los datos.
Indica la relacion lineal entre los parametros. Un valor de correlación alto y positivo indica que los elementos miden la misma característica. Si los elementos no están altamente correlacionados, entonces los elementos pueden medir diferentes características.

In [39]:
corr_matrix = BD.corr()
corr_matrix.style.background_gradient(cmap='coolwarm')

,Id,Lifetime,user_intake,user_no_outgoing_activity_in_days,user_has_outgoing_calls,user_has_outgoing_sms,user_use_gprs,user_does_reload,reloads_inactive_days,reloads_count,calls_outgoing_count,calls_outgoing_inactive_days,calls_outgoing_to_onnet_count,calls_outgoing_to_onnet_inactive_days,calls_outgoing_to_offnet_count,calls_outgoing_to_offnet_inactive_days,calls_outgoing_to_abroad_count,calls_outgoing_to_abroad_inactive_days,sms_outgoing_count,sms_outgoing_inactive_days,sms_outgoing_to_onnet_count,sms_outgoing_to_onnet_inactive_days,sms_outgoing_to_offnet_count,sms_outgoing_to_offnet_inactive_days,sms_outgoing_to_abroad_count,sms_outgoing_to_abroad_inactive_days,sms_incoming_count,sms_incoming_from_abroad_count,gprs_session_count,gprs_inactive_days,last_100_reloads_count,last_100_sms_outgoing_count,last_100_sms_outgoing_to_onnet_count,last_100_sms_outgoing_to_offnet_count,last_100_sms_outgoing_to_abroad_count,Retiro
Id,1.000000,0.474652,0.254815,0.074392,-0.387371,-0.336866,-0.002244,0.129659,-0.097850,0.009357,-0.074943,0.428080,-0.032266,0.428080,-0.089532,0.428080,-0.015849,0.428080,-0.075812,0.393313,-0.034941,0.393313,-0.040441,0.393313,-0.042440,0.393313,-0.111731,-0.062749,0.006528,0.047406,-0.091240,-0.095065,-0.045914,-0.055299,-0.039027,0.357816
Lifetime,0.474652,1.000000,0.124386,0.314474,-0.648572,-0.534862,-0.081538,0.074515,0.020458,-0.089928,-0.256309,0.721231,-0.061830,0.721231,-0.227112,0.721231,-0.056923,0.721231,-0.175824,0.674132,-0.087950,0.674132,-0.152317,0.674132,-0.066045,0.674132,-0.263147,-0.123485,-0.025409,0.165816,-0.205796,-0.190805,-0.102034,-0.168767,-0.059954,0.529719
user_intake,0.254815,0.124386,1.000000,-0.041898,0.091288,0.090708,0.091401,0.175389,-0.121203,0.096087,0.009693,-0.066725,0.006645,-0.066725,-0.003230,-0.066725,0.003873,-0.066725,0.007505,-0.050974,0.004494,-0.050974,0.016634,-0.050974,0.004650,-0.050974,0.078559,-0.019011,0.044884,-0.033978,-0.013298,-0.050984,-0.027395,-0.040466,-0.015998,-0.014725
user_no_outgoing_activity_in_days,0.074392,0.314474,-0.041898,1.000000,-0.460601,-0.285890,-0.049772,-0.259467,0.371698,-0.161868,-0.130965,0.513585,-0.030984,0.513585,-0.113782,0.513585,-0.027513,0.513585,-0.088699,0.331071,-0.045276,0.331071,-0.076123,0.331071,-0.032738,0.331071,-0.144910,-0.056053,-0.015932,0.060536,-0.129750,-0.090831,-0.048533,-0.079034,-0.028691,0.222895
user_has_outgoing_calls,-0.387371,-0.648572,0.091288,-0.460601,1.000000,0.550550,0.098340,0.091328,-0.181520,0.168472,0.273958,-0.840736,0.065029,-0.840736,0.237592,-0.840736,0.057714,-0.840736,0.178841,-0.615234,0.089569,-0.615234,0.152671,-0.615234,0.064686,-0.615234,0.292305,0.111586,0.029535,-0.149894,0.220252,0.179995,0.095250,0.155355,0.054983,-0.553790
user_has_outgoing_sms,-0.336866,-0.534862,0.090708,-0.285890,0.550550,1.000000,0.148630,0.269465,-0.330388,0.291707,0.321280,-0.500844,0.079794,-0.500844,0.292777,-0.500844,0.072662,-0.500844,0.303962,-0.835384,0.155116,-0.835384,0.261107,-0.835384,0.112481,-0.835384,0.366230,0.138057,0.049119,-0.186813,0.310892,0.293820,0.159339,0.253473,0.092850,-0.490214
user_use_gprs,-0.002244,-0.081538,0.091401,-0.049772,0.098340,0.148630,1.000000,0.105847,-0.097144,0.147496,0.107285,-0.087369,0.023393,-0.087369,0.081807,-0.087369,0.017903,-0.087369,0.128582,-0.131679,0.070539,-0.131679,0.101248,-0.131679,0.039593,-0.131679,0.185276,0.022293,0.309446,-0.745309,0.144041,0.116666,0.067636,0.091714,0.023950,-0.078414
user_does_reload,0.129659,0.074515,0.175389,-0.259467,0.091328,0.269465,0.105847,1.000000,-0.700629,0.616221,0.291378,0.048187,0.073570,0.048187,0.270856,0.048187,0.059637,0.048187,0.237657,-0.150940,0.101987,-0.150940,0.192064,-0.150940,0.076252,-0.150940,0.270219,0.059462,0.049952,-0.070571,0.375547,0.216213,0.100515,0.173922,0.057913,-0.043394
reloads_inactive_days,-0.097850,0.020458,-0.121203,0.371698,-0.181520,-0.330388,-0.097144,-0.700629,1.000000,-0.436336,-0.262388,0.045696,-0.066032,0.045696,-0.241897,0.045696,-0.056629,0.045696,-0.205876,0.27785

La columna de salida "Retiro" es la que se quiere predecir, por ende ordenamos los valores de Correlacion entre la salida y las demas variables

In [40]:
corr_matrix["Retiro"].sort_values(ascending=False)

Retiro                                    1.000000
calls_outgoing_to_offnet_inactive_days    0.546173
calls_outgoing_inactive_days              0.546173
calls_outgoing_to_onnet_inactive_days     0.546173
calls_outgoing_to_abroad_inactive_days    0.546173
sms_outgoing_to_abroad_inactive_days      0.540235
sms_outgoing_to_offnet_inactive_days      0.540235
sms_outgoing_to_onnet_inactive_days       0.540235
sms_outgoing_inactive_days                0.540235
Lifetime                                  0.529719
Id                                        0.357816
user_no_outgoing_activity_in_days         0.222895
reloads_inactive_days                     0.132148
gprs_inactive_days                        0.128939
user_intake                              -0.014725
gprs_session_count                       -0.022318
calls_outgoing_to_abroad_count           -0.040320
user_does_reload                         -0.043394
last_100_sms_outgoing_to_abroad_count    -0.051195
calls_outgoing_to_onnet_count  

En la celda anterior podemos ver la correlacion de todas la variables del dataser con la variable etiqueta (Retiro) que se busca predecir con el modelo. Una correlacion positiva nos indica que esas dos variables aumentan o disminuyen en una misma direccion, mientras que una correlacion negativa indica que mientras una aumenta la otra disminuye. 

Con base en esto y por pura inferencia estadistica podriamos decir que las variables que tienen mas influencia sobre la desercion de los clientes son las siguientes:

* user_has_outgoing_calls       --->           -0.553790
* user_has_outgoing_sms                    -0.490214
* calls_outgoing_to_offnet_inactive_days --->   0.546173
* calls_outgoing_inactive_days       --->       0.546173
* calls_outgoing_to_onnet_inactive_days  --->   0.546173
* calls_outgoing_to_abroad_inactive_days  --->  0.546173
* sms_outgoing_to_abroad_inactive_days  --->    0.540235
* sms_outgoing_to_offnet_inactive_days   --->   0.540235
* sms_outgoing_to_onnet_inactive_days   --->    0.540235
* sms_outgoing_inactive_days        --->        0.540235
* Lifetime                         --->         0.529719
